## Garbage Collection

### Introduction

Manual garbage collection:
In C: malloc then free. Basically reserve the memory and you have to free the physical memory at the end of the program in order to avoid resource occupation. Failure to do so might increase the chance of memory leak and eventually run out of available memory. Automatic Memory Management System will work out when finished using memory and give memory back to the OS. That means only malloc is needed without calling free. 

Below is an example program in C that allocates memory for one integer as pointer then recycles and frees that space at the end:


In [2]:
%%writefile example.c
#include <stdio.h>
#include <stdlib.h> // Required for malloc and free

int main() {
    int *ptr = malloc(sizeof(int)); // Allocate memory for one integer

    if (ptr == NULL) { // Check if the memory allocation failed
        fprintf(stderr, "Memory allocation failed\n");
        return 1; // Return with error code
    }

    *ptr = 10; // Assign a value to the allocated memory
    printf("Value of *ptr: %d\n", *ptr); // Print the value

    free(ptr); // Free the allocated memory
    ptr = NULL; // Avoid dangling pointer by setting it to NULL

    return 0; // Return success
}


Writing example.c


In [3]:
!gcc example.c -o example

In [4]:
!./example

Value of *ptr: 10


Almost all modern programming languages make use of `dynamic memory allocation`. This allows objects to be allocated and deallocated even if their total size was not known at the time that the program was compiled, and if their lifetime may exceed that of the subroutine activation that allocated them. A dynamically allocated object is stored in a heap, rather than on the stack (in the activation record or stack frame of the procedure that allocated it) or statically (whereby the name of an object is bound to a storage location known at compile or link time). 

Heap allocation is particularly important because it allows the programmer to:

    1. choose dynamically the size of new objects (thus avoiding program failure through exceeding hard-coded limits on arrays);
    2. deﬁne and use recursive data structures such as lists, trees and maps;
    3. return newly created objects to the parent procedure (allowing, for example, factorymethods);
    4. return a function as the result of another function (for example, closures or suspensionsin functional languages, or futures or streams in concurrent ones).

The goal of an ideal garbage collector is to reclaim the space used by every object that will no longer be used by the program. Any automatic memory management system has three tasks and they're not independent:

    1. to allocate space for new objects;
    2. to identify live objects as well as dead ones; 
    3. to reclaim the space occupied by dead objects.

### Some Terminologies

`Roots:` Typically refer to objects that are directly accessible or referenced by the interpreter, e.g registers, thread stacks, global variables. These objects serve as starting points for the garbage collector to traverse the object graph and determine which objects are reachable and which ones are unreachable for the purpose of memory management. `Any object reference from a variable is a root.`

`Dangling references: ` Pointers or references in a program that point to memory locations that have been deallocated, released, or otherwise invalidated. In simpler terms, they are references that are left "hanging" without a valid target. 

Here's 3 possible scenarios:

1.Deallocated Memory: If memory allocated for an object is freed or deallocated, but there are still pointers or references pointing to that memory location, those pointers become dangling references.

2.Out-of-Scope References: In languages like C or C++, if a pointer to an object is stored in a local variable within a function and that function returns, the local variable goes out of scope, but if the pointer is still accessed or used elsewhere, it becomes a dangling reference because it points to memory that may have been reused for other purposes.

3.Object Ownership: In languages without garbage collection, if an object is manually deallocated while other parts of the program still hold references to it, those references become dangling.

`Strong Reference:` A strong reference is a direct reference to an object that prevents it from being garbage-collected. As long as there is at least one strong reference to an object, the garbage collector considers it "alive" and will not reclaim its memory. Most references in programming are strong references by default. For example, in languages like Java and C#, simply declaring and assigning an object to a variable creates a strong reference. These references ensure that the object remains in memory throughout its use, preventing it from being collected prematurely.

`Weak Reference:` A weak reference, on the other hand, does not prevent the garbage collector from reclaiming the object it points to. This means that an object with only weak references is eligible for garbage collection and can be collected at any time when the system runs a garbage collection cycle. Weak references are useful in situations where you need a reference to an object without preventing it from being garbage-collected, which is especially valuable in scenarios involving caching, listeners, or large data structures where memory management is crucial.

`Mutators:` These are essentially the application threads that run the main logic of the program. A mutator modifies the heap memory by allocating new objects and potentially making some objects unreachable. The term "mutator" is used because these threads mutate the state of the memory, continually changing which objects are accessible and which are not.

`Collectors:` These are part of the garbage collection system. A collector's job is to identify memory that is no longer in use and can be reclaimed. The collector operates in its own thread or set of threads separate from those of the application. It scans the memory, marks unused objects, and eventually deallocates or recycles this memory, making it available for future use.





### Reference Counting

When allocating memory, put a little counter on the object header saying how many parts of the program are still using this. Local variables and formal parameters cause decrements at the end of 
their lifetime. `Assigning nil causes a decrement.` Reference counting maintains a simple invariant: an object is presumed to be live if and only if the number of references to that object is greater than zero. Reference counting therefore associates a reference count with each object managed; typically this count is stored as an additional slot in the object's header.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="1.png"></img> 

In contrast to the tracing algorithms like mark&sweep, **Algorithm 5.1** redefines all pointer `Read` and `Write` operations in order to manipulate reference counts. Even non-destructive operations such as iteration require the reference counts of each element in the list to be incremented and then decremented as a pointer moves across a data structure such as a list.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="14.png"></img> 
This is a `direct collection method`, direct algorithms determine the liveness of an object from the object alone, without recourse to tracing.

This method could have some problems: 

`1. Add counter and forgot to decrement: ` The most direct consequence of not decrementing a reference count when you should is a memory leak. If a reference count is never decremented properly after the last usage of the object, the counter never reaches zero. This prevents the garbage collector or memory management system from reclaiming the object’s allocated memory, even though it's no longer in use. Over time, these leaks can consume significant amounts of memory, degrading system performance and stability. E.g: Complex Control Flows, Exception Handling, Manual Management.

`2. Performance issues: ` From a performance point of view, it is particularly undesirable to add overhead to operations that manipulate registers or thread stack slots. For this reason alone, this naive algorithm is impractical for use as a general purpose, high volume, high performance memory manager.

`3. Can’t deal with cycles:` Cycles occur when two or more objects reference each other in a circular manner. In such cases, even though there are no external references to the objects involved in the cycle, their reference counts will never reach zero because they are still referencing each other. Because of this, memory occupied by cyclically referenced objects will never be deallocated using reference counting alone, leading to what's known as a memory leak. <img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="2.png"></img>  

Applications of Reference Counting:

    better suited for real-time programming;
    used in distributed systems, where tracing all pointers is impractical;
    used in the Unix file system;
    used in the Swift language: strong, weak/unowned references are distinguished

### Deferred Reference Counting

Manipulating all reference counts is expensive compared with the cost to the mutator of simple tracing algorithms. Most high-performance reference counting systems use deferred reference counting. The overwhelming majority of pointer loads are to local and temporary variables, that is, to registers or stack slots. A way to reduce this overhead is to keep only an approximate count: pointer stores to local variables do not update the count, `only stores to the heap do`; the count reflects then the number of heap pointers only. The deferred aspect means that updates to the reference count and the associated garbage collection are not performed immediately but are `deferred` to a later time, which can help in optimizing performance especially in `concurrent execution environments`. If the count reaches zero, then the stack has to be scanned to check if local variables are pointing to the object before it can be recycled. This can be deferred by placing objects with zero count in a zero count table first and scanning that table periodically as shown in **Algorithm 5.2**.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="16.png"></img>

### Mark & Sweep

Garbage collection works in two phases(Mark & Sweep). We assume that each object 
on the heap has one spare bit for marking and unmarking:

`Mark Phase:`
The mark phase involves traversing the entire object graph starting from the roots (objects directly accessible by the program) and marking each object encountered as reachable.

Objects that are reachable are typically marked using a flag or a bit in their header to indicate that they are still in use.
During the traversal, the algorithm follows references from one object to another, marking each object as it encounters them. Such a traversal is called `tracing`.
This phase ensures that all objects reachable from the roots are identified and marked as live.

`Sweep Phase:`
The sweep phase involves traversing the entire heap (memory space allocated for objects) and deallocating memory for objects that are not marked as reachable.
The algorithm scans through each memory block, checking whether the corresponding object is marked. If it's marked, it means the object is still in use, so the mark is cleared. If it's not marked, it means the object is no longer reachable and can be safely deallocated.
After sweeping through all memory blocks, the algorithm frees the memory associated with unmarked objects, making it available for future allocations.

The mark-sweep interface with the mutator is very simple. If a thread is unable to allocate a new object, the collector is called and the allocation request is retried (**Algorithm 2.1**). To emphasize that the collector operates in stop-the-world mode, without concurrent execution of the mutator threads, we mark the collect routine with the `atomic` keyword. If there's still insufficient memory available to meet the allocation request, then heap memory is exhausted. Often this is a fatal error. However, in some languages, New may raise an exception in this circumstance that the programmer may be able to catch. If memory can be released by deleting references (for example, to cached data structures which could be recreated later if necessary), then the allocation request could be repeated.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="11.png"></img>  

Before traversing the object graph, the collector must first prime the marker's work list with starting points for the traversal (markFromRoots in **Algorithm 2.2**). Each root object is marked and then added to the work list.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="12.png"></img>  

The sweep phase returns unmarked nodes to the allocator (**Algorithm 2.3**). Typically,the collector sweeps the heap linearly, starting from the bottom, freeing unmarked nodes and resetting the mark bits of marked nodes in preparation for the next collection cycle.Note that we can avoid the cost of resetting the mark bit of live objects if the sense of the bit is switched between one collection and the next.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="13.png"></img>

This is an `indirect collection algorithm`, meaning it doesn't detect garbage per se, but rather identifies all the live objects and then concludes that anything else must be garbage. Note that it needs to recalculate its estimate of the set of live objects at each invocation. Not all garbage collection algorithms behave like this.

Its `drawbacks` include fragmentation of memory and pauses during the garbage collection process, which can impact the performance of real-time or interactive applications. Java could left a pointer behind, that pointer could point to millions of other objects, taking up a lot of space. Also for each recursive call, space on the stack for the activation frame is needed. In case we are reclaiming the memory of a singly linked list, that may need more memory space than the list itself!

### In-Place Garbage Collection (Deutsch-Schorr-Waite)

This algorithm is used to modify pointer fields in objects during a `depth-first search`, making it suitable for garbage collection in languages like C and C++ where manual memory management is necessary. It's a non-recursive algorithm that updates the pointer fields during traversal to maintain a link back to the parent object from the current object. This algorithm utilizes two pointers, named `current` and `previous`, to establish these backward links. Each object in this context is structured with two pointer fields and an additional field that records the index (either 0 or 1) of the pointer that points to the parent.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="3.png"></img>  
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="4.png"></img>  

One big disadvantage of this algorithm is that it can't run `concurrently` with anything. The DSW algorithm fundamentally modifies the data structures it traverses. It temporarily changes pointers in the data structure to point backwards rather than forwards, thereby eliminating the need for an explicit stack. These modifications mean that the data structure is in a transient, inconsistent state during traversal. Running the DSW algorithm concurrently with other operations that either read or modify the same set of nodes can lead to several concurrency hazards:

Race Conditions: If another process is trying to read or modify the data structure while it is being altered by the DSW algorithm, it might encounter incorrect or unexpected pointers, leading to errors or crashes.

Deadlocks: Since the DSW algorithm involves complex manipulations of pointers, running it in parallel with other processes that lock the same resources could lead to deadlocks where each process waits for the other to release resources indefinitely.

`Copying Collection:` The heap is divided into `from-space` and `to-space`. Local and global variables - the roots - point to the from-space:
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="5.png"></img>  
`free` points the next free location; if it reaches `limit`, garbage collection is initiated, copying all reachable objects to the to-space.

<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="6.png"></img> 

`Advantages` of this Coping Collection:

Allocation of new objects only involves incrementing the free pointer; no need to maintain lists of free blocks.

Copying collections also performs a compaction; there is no possibility for fragmentation.

All reachable objects are copied, but depending on the programming language, many objects are not reachable.

`Main Drawback` is that only half of the memory can be used. This can be reduced by dividing the memory into n blocks, selecting one to-space and one from-space among those, and sliding the window. However, objects must be smaller than the block size:

<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="7.png"></img> 

### Mark-Compact Garbage Collection

Mark-compact collection is similar to mark-sweep, but does compaction like copying collection. Initially, starting from the root set, all live objects are `marked`, like in mark-sweep:
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="8.png"></img> 

In a `sweep` phase, the new address of each object is calculated and stored it is forwarding field; the new address is the sum of the sizes of all objects encountered so far.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="9.png"></img> 

In the `third` phase, all pointers are updated to point to the new locations.

In the `fourth` phase, the objects are moved to their new location; the pointers are left unchanged. On this occasion, all objects are unmarked and all forwarding pointers become unused again:
<img style="width:15em;display: block;margin-left: auto;margin-right: auto" src="10.png"></img> 

`Advantage` of this algorithm:
Like copying collection, allocation of new objects only involves incrementing the free pointer; no need to maintain free lists.

One extra pointer for forwarding is needed in each object; mark-sweep `does not need extra space`, copying collections needs `twice as much space`(the `from` and `to` space).

Like with mark-sweep, the marking phase traverses the whole heap; if it is larger than main memory, swapping occurs. Copying collection only traverses live objects; sometimes only 5% are live.

Compaction preserves the original order of objects, unlike copying collection; this supports `locality`(by keeping the original order of objects and reducing the gaps between them, the data structure or memory layout optimizes both spatial and temporal locality) and `better caching`.

Mark-compact tends to accumulate long-lived objects at the `bottom of the heap` and would not move them. Copying collections always moves objects, whether small or large.


### Generational Garbage Collection

Most objects die shortly after they are created: either an object is created as a "local" variable by the program or it's created to store the result of expression evaluation. Almost all objects collected are created since the last collection. `In other words, once an object survives the first collection, it is likely to survive subsequent ones.`

Generational collection divides the heap into a number of generations, say an old and a young generation. Different strategies are used for different generations:

`A tracing collection like mark-compact is used on the old generation.`

`A copying collector is used on the new generation.`

Once the heap is full, a minor collection on the youngest generation is performed; if that does not reclaim sufficient memory, the next older generation is collected.
<img style="width:40em;display: block;margin-left: auto;margin-right: auto" src="15.png"></img> 
The young generation is collected by following the pointers from the root set but ignoring those to the old generation.

However, pointers from the old generation to the new generation can exist and have to be treated as belonging to the root set. Several options exists:

`Remembered set:` whenever a pointer to a new object is stored in an old object, the pointer is added to a remembered set; whenever a new object becomes old, all its pointers to new objects are placed in the remembered set as well.

`Card marking:` the heap is divided into cards of equal size and a vector with a bit for every card is maintained. Whenever a pointer is stored in an object, the bit for the location of the pointer is set. A minor collection will check all the marked cards in the old generation. (Used by JDK 1.4.)

`Page marking:` if the card size is the page size of the virtual memory, the dirty bit of the page can be used for marking; this assumes that the dirty bit is available to user programs!